In [1]:
import os
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [2]:
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

In [3]:
from models.RNN import SimpleRNN
from utils.simulation import Simulation
from training.train import train_and_validate, evaluate_model

In [4]:
# loading the data
wd_datapath = str(root)+"/data/normalized_data/tra_val"
# dem_datapath = str(root)+"/datanormalized_data/tra_val"
grid_size = 64
sim_number = 1


# select one simulation for training
sim = Simulation.load_simulations(save_folder=wd_datapath,
                                  sim_amount = 10,
                                  number_grids=grid_size)

timestep = len(sim.wd)

plt.imshow(sim.wd[-1])

FileNotFoundError: C:\Users\sagi8\Documents\GitHub\DS-AI/data/normalized_data/tra_val\VX\vx_11_b.txt not found.

### Load all the simulations for training  

In [ ]:

# Reshape the data into into the 2D array with
# each row representing the pixels (each pixel is regarded as a new variable)
# each column representing the timestep (condider the temporal features as the other dimension of images)
def dataconvert(meshdata,timestep,gridsize):
    tempCNN_array = np.zeros((gridsize*gridsize,timestep))
    for i in range(timestep):
        tempCNN_array[:,i] = meshdata[i].reshape(-1)




### Use the Unet Encoder-Decoder CNN to train the model  

In [ ]:
from RNN import CNNBlock
from RNN import CNN
from training.train import train_and_validate
from training.train import evaluate_model
node_features = 1 # Water Depth

model = CNN(node_features=node_features,
            n_downsamples=4,
            initial_hid_dim=32,
            batch_norm=True, bias=True)

In [ ]:
display(model)

### Define the training parameter, the optimizer, and the dataloader

In [ ]:
# Set training parameters
learning_rate = 0.001
batch_size = 32
num_epochs = 20

# Create the optimizer to train the neural network via back-propagation
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

# Create the training and validation dataloaders to "feed" data to the model in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(normalized_test_dataset, batch_size=batch_size, shuffle=False)

### Training and Validating 

In [ ]:
train_losses = []
val_losses = []

for epoch in range(1, num_epochs+1):
    # Model training
    train_loss = train_epoch(model, train_loader, optimizer, device=device)

    # Model validation
    val_loss = evaluation(model, val_loader, device=device)

    if epoch == 1:
        best_loss = val_loss

    if val_loss<=best_loss:
        best_model = copy.deepcopy(model)
        best_loss = val_loss
        best_epoch = epoch

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    if epoch%10 == 0:
        print("epoch:",epoch, "\t training loss:", np.round(train_loss,4),
                            "\t validation loss:", np.round(val_loss,4))

model = copy.deepcopy(best_model)

In [ ]:
test_loss = evaluation(model, test_loader, device=device)
print(test_loss)

### Visualisation of results

In [ ]:
plt.plot(train_losses, label='Training')
plt.plot(val_losses, label='Validation')
plt.yscale('log')
plt.title('Losses')
plt.xlabel('Epochs')
plt.legend()
plt.show()